In [1]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *
how_many=64
# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=5
classes=10
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:how_many]

label=np.full((how_many),9)
for i in range(classes-1):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    label_temp=np.full((how_many),i)
    

    all=np.concatenate((x[:how_many],all))
    label=np.concatenate((label_temp,label))
   
random_index = np.random.permutation(how_many*classes)
all=torch.tensor(all).to(device)
label=torch.tensor(label).to(device)
# Mischiare i campioni e le etichette utilizzando l'indice di permutazione casuale

#data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

qc_array=np.array([0,32,56,62,63])
min_array=np.array([0.05,0.01,0.005])
layer_array=np.array([20]) 
num_batch=classes
print(NUM_QUBITS)
print(T)
zero = torch.zeros(BATCH_SIZE, 2**NUM_QUBITS-ld_dim).to(device)

for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=classes
                tot_loss=0
                random_index = np.random.permutation(how_many*classes)

                # Mischiare i campioni e le etichette utilizzando l'indice di permutazione casuale
                mnist_images = all[random_index]
                labels = label[random_index]
                
                for batch_indx in range(int(how_many*classes/BATCH_SIZE)): #qua dovrebbe essere il numero di batch, ma visto che sono uguali al numero di classi metto il numero di clasii
                    loss_batch=0
                    image_batch=mnist_images[BATCH_SIZE*batch_indx:BATCH_SIZE*(batch_indx+1)]
                    label_batch=labels[BATCH_SIZE*batch_indx:BATCH_SIZE*(batch_indx+1)]
                    
                    
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    
                
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    for j in range(classes):
                        
                        indici_classe = (label_batch == j).nonzero().squeeze()
                        
                        img_batch_temp=input_batch[indici_classe]
                        target_batch_temp=input_batch[indici_classe]
                        
                        zero_input = torch.zeros((len(img_batch_temp), 2**NUM_QUBITS), dtype=torch.complex64)
                        zero_target = torch.zeros((len(img_batch_temp), 2**NUM_QUBITS), dtype=torch.complex64)
                        

                        zero_input[:,j*ld_dim:(j+1)*ld_dim]=img_batch_temp
                        
                        zero_target[:,j*ld_dim:(j+1)*ld_dim]=target_batch_temp
                        
                        
                        loss_label = loss_fn_aq(qc,theta_1,n_layer, zero_input, zero_target,j)
                        #if j==9: print(f'loss{loss_label}, label={j}')
                        loss_batch+=loss_label/classes
                        
                    
                    
                    tot_loss+=loss_batch.item()/num_batch
                    loss_batch.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                    
                    
  


                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss)
                if tot_loss< best_loss:
                    best_loss=tot_loss

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {tot_loss:.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer} QC{qc}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'thetas/all_thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}_cond_all.npy',trained_thetas_1)
            np.save(f'losses/all_loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}_cond_all.npy',loss_history)

6
5
(6,)
0
T=5 Epoch: 1/30 - Loss: 0.1336 b_loss=0.1336 - T: 6.92s/epoch ,tempo_previto=51.75 min0.05 nl20 QC0
1
T=5 Epoch: 2/30 - Loss: 0.0945 b_loss=0.0945 - T: 6.92s/epoch ,tempo_previto=51.68 min0.05 nl20 QC0
2
T=5 Epoch: 3/30 - Loss: 0.0692 b_loss=0.0692 - T: 7.01s/epoch ,tempo_previto=52.23 min0.05 nl20 QC0
3
T=5 Epoch: 4/30 - Loss: 0.0532 b_loss=0.0532 - T: 6.99s/epoch ,tempo_previto=51.95 min0.05 nl20 QC0
4
T=5 Epoch: 5/30 - Loss: 0.0454 b_loss=0.0454 - T: 7.15s/epoch ,tempo_previto=53.01 min0.05 nl20 QC0
5
T=5 Epoch: 6/30 - Loss: 0.0411 b_loss=0.0411 - T: 7.18s/epoch ,tempo_previto=53.10 min0.05 nl20 QC0
6
T=5 Epoch: 7/30 - Loss: 0.0370 b_loss=0.0370 - T: 7.00s/epoch ,tempo_previto=51.70 min0.05 nl20 QC0
7
T=5 Epoch: 8/30 - Loss: 0.0344 b_loss=0.0344 - T: 7.17s/epoch ,tempo_previto=52.79 min0.05 nl20 QC0
8
T=5 Epoch: 9/30 - Loss: 0.0324 b_loss=0.0324 - T: 7.14s/epoch ,tempo_previto=52.47 min0.05 nl20 QC0
9
T=5 Epoch: 10/30 - Loss: 0.0285 b_loss=0.0285 - T: 7.26s/epoch ,tempo_p

T=5 Epoch: 20/30 - Loss: 0.0181 b_loss=0.0181 - T: 7.29s/epoch ,tempo_previto=15.79 min0.005 nl20 QC0
20
T=5 Epoch: 21/30 - Loss: 0.0174 b_loss=0.0174 - T: 7.23s/epoch ,tempo_previto=15.54 min0.005 nl20 QC0
21
T=5 Epoch: 22/30 - Loss: 0.0180 b_loss=0.0174 - T: 7.10s/epoch ,tempo_previto=15.15 min0.005 nl20 QC0
22
T=5 Epoch: 23/30 - Loss: 0.0177 b_loss=0.0174 - T: 7.22s/epoch ,tempo_previto=15.29 min0.005 nl20 QC0
23
T=5 Epoch: 24/30 - Loss: 0.0174 b_loss=0.0174 - T: 7.18s/epoch ,tempo_previto=15.07 min0.005 nl20 QC0
24
T=5 Epoch: 25/30 - Loss: 0.0173 b_loss=0.0173 - T: 7.26s/epoch ,tempo_previto=15.13 min0.005 nl20 QC0
25
T=5 Epoch: 26/30 - Loss: 0.0171 b_loss=0.0171 - T: 7.03s/epoch ,tempo_previto=14.52 min0.005 nl20 QC0
26
T=5 Epoch: 27/30 - Loss: 0.0172 b_loss=0.0171 - T: 7.00s/epoch ,tempo_previto=14.35 min0.005 nl20 QC0
27
T=5 Epoch: 28/30 - Loss: 0.0169 b_loss=0.0169 - T: 7.30s/epoch ,tempo_previto=14.85 min0.005 nl20 QC0
28
T=5 Epoch: 29/30 - Loss: 0.0177 b_loss=0.0169 - T: 7.18

T=5 Epoch: 9/30 - Loss: 0.0296 b_loss=0.0296 - T: 7.28s/epoch ,tempo_previto=13.47 min0.005 nl20 QC32
9
T=5 Epoch: 10/30 - Loss: 0.0256 b_loss=0.0256 - T: 7.20s/epoch ,tempo_previto=13.20 min0.005 nl20 QC32
10
T=5 Epoch: 11/30 - Loss: 0.0224 b_loss=0.0224 - T: 7.16s/epoch ,tempo_previto=13.00 min0.005 nl20 QC32
11
T=5 Epoch: 12/30 - Loss: 0.0215 b_loss=0.0215 - T: 7.17s/epoch ,tempo_previto=12.91 min0.005 nl20 QC32
12
T=5 Epoch: 13/30 - Loss: 0.0200 b_loss=0.0200 - T: 7.18s/epoch ,tempo_previto=12.81 min0.005 nl20 QC32
13
T=5 Epoch: 14/30 - Loss: 0.0192 b_loss=0.0192 - T: 7.20s/epoch ,tempo_previto=12.72 min0.005 nl20 QC32
14
T=5 Epoch: 15/30 - Loss: 0.0181 b_loss=0.0181 - T: 6.98s/epoch ,tempo_previto=12.21 min0.005 nl20 QC32
15
T=5 Epoch: 16/30 - Loss: 0.0175 b_loss=0.0175 - T: 7.39s/epoch ,tempo_previto=12.81 min0.005 nl20 QC32
16
T=5 Epoch: 17/30 - Loss: 0.0164 b_loss=0.0164 - T: 7.19s/epoch ,tempo_previto=12.34 min0.005 nl20 QC32
17
T=5 Epoch: 18/30 - Loss: 0.0170 b_loss=0.0164 - 

T=5 Epoch: 28/30 - Loss: 0.0210 b_loss=0.0208 - T: 7.23s/epoch ,tempo_previto=25.56 min0.01 nl20 QC56
28
T=5 Epoch: 29/30 - Loss: 0.0210 b_loss=0.0208 - T: 7.23s/epoch ,tempo_previto=25.41 min0.01 nl20 QC56
29
T=5 Epoch: 30/30 - Loss: 0.0200 b_loss=0.0200 - T: 7.06s/epoch ,tempo_previto=24.71 min0.01 nl20 QC56
(6,)
0
T=5 Epoch: 1/30 - Loss: 0.1496 b_loss=0.1496 - T: 6.97s/epoch ,tempo_previto=10.34 min0.005 nl20 QC56
1
T=5 Epoch: 2/30 - Loss: 0.1171 b_loss=0.1171 - T: 7.20s/epoch ,tempo_previto=10.56 min0.005 nl20 QC56
2
T=5 Epoch: 3/30 - Loss: 0.0900 b_loss=0.0900 - T: 7.06s/epoch ,tempo_previto=10.24 min0.005 nl20 QC56
3
T=5 Epoch: 4/30 - Loss: 0.0669 b_loss=0.0669 - T: 7.31s/epoch ,tempo_previto=10.47 min0.005 nl20 QC56
4
T=5 Epoch: 5/30 - Loss: 0.0525 b_loss=0.0525 - T: 7.47s/epoch ,tempo_previto=10.59 min0.005 nl20 QC56
5
T=5 Epoch: 6/30 - Loss: 0.0428 b_loss=0.0428 - T: 7.22s/epoch ,tempo_previto=10.11 min0.005 nl20 QC56
6
T=5 Epoch: 7/30 - Loss: 0.0382 b_loss=0.0382 - T: 7.20s/e

T=5 Epoch: 17/30 - Loss: 0.0235 b_loss=0.0230 - T: 7.52s/epoch ,tempo_previto=24.18 min0.01 nl20 QC62
17
T=5 Epoch: 18/30 - Loss: 0.0224 b_loss=0.0224 - T: 7.14s/epoch ,tempo_previto=22.86 min0.01 nl20 QC62
18
T=5 Epoch: 19/30 - Loss: 0.0230 b_loss=0.0224 - T: 7.07s/epoch ,tempo_previto=22.49 min0.01 nl20 QC62
19
T=5 Epoch: 20/30 - Loss: 0.0220 b_loss=0.0220 - T: 7.31s/epoch ,tempo_previto=23.14 min0.01 nl20 QC62
20
T=5 Epoch: 21/30 - Loss: 0.0214 b_loss=0.0214 - T: 7.06s/epoch ,tempo_previto=22.25 min0.01 nl20 QC62
21
T=5 Epoch: 22/30 - Loss: 0.0214 b_loss=0.0214 - T: 7.25s/epoch ,tempo_previto=22.72 min0.01 nl20 QC62
22
T=5 Epoch: 23/30 - Loss: 0.0218 b_loss=0.0214 - T: 7.25s/epoch ,tempo_previto=22.59 min0.01 nl20 QC62
23
T=5 Epoch: 24/30 - Loss: 0.0214 b_loss=0.0214 - T: 7.29s/epoch ,tempo_previto=22.59 min0.01 nl20 QC62
24
T=5 Epoch: 25/30 - Loss: 0.0209 b_loss=0.0209 - T: 7.08s/epoch ,tempo_previto=21.83 min0.01 nl20 QC62
25
T=5 Epoch: 26/30 - Loss: 0.0209 b_loss=0.0209 - T: 7.45

T=5 Epoch: 6/30 - Loss: 0.0473 b_loss=0.0473 - T: 7.37s/epoch ,tempo_previto=21.36 min0.01 nl20 QC63
6
T=5 Epoch: 7/30 - Loss: 0.0446 b_loss=0.0446 - T: 7.59s/epoch ,tempo_previto=21.89 min0.01 nl20 QC63
7
T=5 Epoch: 8/30 - Loss: 0.0412 b_loss=0.0412 - T: 7.46s/epoch ,tempo_previto=21.37 min0.01 nl20 QC63
8
T=5 Epoch: 9/30 - Loss: 0.0397 b_loss=0.0397 - T: 7.34s/epoch ,tempo_previto=20.93 min0.01 nl20 QC63
9
T=5 Epoch: 10/30 - Loss: 0.0375 b_loss=0.0375 - T: 7.21s/epoch ,tempo_previto=20.42 min0.01 nl20 QC63
10
T=5 Epoch: 11/30 - Loss: 0.0367 b_loss=0.0367 - T: 7.14s/epoch ,tempo_previto=20.12 min0.01 nl20 QC63
11
T=5 Epoch: 12/30 - Loss: 0.0353 b_loss=0.0353 - T: 7.48s/epoch ,tempo_previto=20.94 min0.01 nl20 QC63
12
T=5 Epoch: 13/30 - Loss: 0.0343 b_loss=0.0343 - T: 7.49s/epoch ,tempo_previto=20.86 min0.01 nl20 QC63
13
T=5 Epoch: 14/30 - Loss: 0.0339 b_loss=0.0339 - T: 7.28s/epoch ,tempo_previto=20.13 min0.01 nl20 QC63
14
T=5 Epoch: 15/30 - Loss: 0.0324 b_loss=0.0324 - T: 7.80s/epoch 